In [37]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125315")
exp = Experiment(workspace=ws, name="quick-starts-ws-124627")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125315
Azure region: southcentralus
Subscription id: dafd62fe-51d3-4671-af50-6bac2f001b5d
Resource group: aml-quickstarts-125315


In [38]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###cpu_cluster_name = "cpucluster"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
Cpu_Cluster = ComputeTarget.create(ws, "CpuCluster", compute_config)

Cpu_Cluster.wait_for_completion(show_output=True)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [64]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

import os


# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C":uniform(0.05,1),
        "--max_iter":choice(20,40,60.80,100)
        
    
    }
) ### YOUR CODE HERE ###


# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
from azureml.core import ScriptRunConfig
from azureml.core import Environment

script='train.py'
est=SKLearn(source_directory='.',entry_script=script,compute_target=Cpu_Cluster)



# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4
                                    )
                              ### YOUR CODE HERE ###

In [65]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run=exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show() #Run history widget
hyperdrive_run.wait_for_completion(show_output=True)
#best_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['Arguments']

#print('\n Accuracy:', best_run_metrics['accuracy'])


### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_07bcc8eb-4831-469d-8c74-028ef25c2461
Web View: https://ml.azure.com/experiments/quick-starts-ws-124627/runs/HD_07bcc8eb-4831-469d-8c74-028ef25c2461?wsid=/subscriptions/dafd62fe-51d3-4671-af50-6bac2f001b5d/resourcegroups/aml-quickstarts-125315/workspaces/quick-starts-ws-125315

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-09T00:05:37.792559][API][INFO]Experiment created<END>\n""<START>[2020-11-09T00:05:38.262304][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-11-09T00:05:39.0371804Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-09T00:05:38.507407][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_07bcc8eb-4831-469d-8c74-028ef25c2461
Web View: https://ml.azure.com/experiments/quick-starts-ws-124627/runs/HD_07bcc8eb-4831-469d-8c74-028ef25c2461?wsid=/subsc

{'runId': 'HD_07bcc8eb-4831-469d-8c74-028ef25c2461',
 'target': 'CpuCluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-09T00:05:37.547574Z',
 'endTimeUtc': '2020-11-09T00:12:32.10102Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6a62b200-ff70-48f7-b665-df0f78bbade5',
  'score': '0.9072837632776934',
  'best_child_run_id': 'HD_07bcc8eb-4831-469d-8c74-028ef25c2461_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg125315.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_07bcc8eb-4831-469d-8c74-028ef25c2461/

In [69]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id)
print(best_run_metrics)
#model_hd=best_run.register_model(model_name='log_reg_with_hyperdrive',outputpath='outputs/model_hd')

Best Run Id:  HD_07bcc8eb-4831-469d-8c74-028ef25c2461_0
{'Regularization Strength:': 0.5858899974655808, 'Max iterations:': 20, 'Accuracy': 0.9072837632776934}


In [42]:
from azureml.data.dataset_factory import TabularDatasetFactory


# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [43]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###

In [49]:
#splitting the data
from sklearn.model_selection import train_test_split  
import pandas as pd
x_train,x_test,y_train,y_test= train_test_split(x,y, train_size=0.8, test_size=0.2, random_state=42) 
train_data=pd.concat((x_train,y_train),axis=1)


In [60]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=2)

In [63]:
# Submit your automl run
#ws = Workspace.from_config()
run_ml = experiment.submit(automl_config, show_output=True)

### YOUR CODE HERE ###

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
cryptography<=3.1.1/cryptography 3.2
zipp<=3.3.1/zipp 3.4.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nRequired version/Installed version\ncryptography<=3.1.1/cryptography 3.2\nzipp<=3.3.1/zipp 3.4.0",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
         

In [ ]:
# Retrieve and save your best automl model.
model_automl=run.register_model(model_name='log_reg_automl',outputpath='outputs/model_automl')
### YOUR CODE HERE ###